<a href="https://colab.research.google.com/github/abar-1/SDR-ML-Project/blob/linearActivation/QAMReceiverLinearActV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function to convert binary data to QAM16 constellations

In [ ]:
pip install tensorflow

16QAM Constellation Modulator

In [15]:
import numpy as np
"""
Used https://dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/ for mappings. When tested, the function works and
the binary is correctly mapped to its correspondingcomplex number based on the constellation. To check, uncomment
the last line in the cell which prints a test run of the function.
"""


def modulator(binary_data, M):
    k = int(np.log2(M))

    # defining the real and imaginary PAM constellation for 16-QAM
    alphaRe = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)
    alphaIm = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)

    # taking b0b1 for real
    ipDecRe = np.array([int(''.join(map(str, b[:k//2])), 2) for b in binary_data])
    ipGrayDecRe = ipDecRe ^ (ipDecRe >> 1)

    # taking b2b3 for imaginary
    ipDecIm = np.array([int(''.join(map(str, b[k//2:])), 2) for b in binary_data])
    ipGrayDecIm = ipDecIm ^ (ipDecIm >> 1)

    # mapping the Gray coded symbols into constellation
    modRe = alphaRe[ipGrayDecRe]
    modIm = alphaIm[ipGrayDecIm]

    # complex constellation
    mod = modRe + 1j * modIm

    return mod

def generate_qam_symbols(M=16, num_symbols=1000):
    # 4 bits per symbol
    k = int(np.log2(M))

    # Generate random binary data
    random_bits = np.random.randint(0, 2, num_symbols * k)

    # Reshape to match modulator input
    binary_data = random_bits.reshape((-1, k))

    # Modulate Binary Data
    modulated_symbols = modulator(binary_data, M)

    return modulated_symbols, binary_data
#print(generate_qam_symbols(16,5))

Adding noise to 16QAM signals

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def add_awgn(qam_symbols, snr_db):
    """
    Adds Additive White Gaussian Noise (AWGN) to QAM symbols.

    Parameters:
        qam_symbols (numpy array): The transmitted QAM symbols (complex numbers).
        snr_db (float): Signal-to-noise ratio in dB.

    Returns:
        numpy array: Noisy QAM symbols.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(qam_symbols) ** 2)

    # Compute noise power based on SNR (convert dB to linear scale)
    noise_power = signal_power / (10 ** (snr_db / 10))

    # Generate AWGN noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*qam_symbols.shape) + 1j * np.random.randn(*qam_symbols.shape))

    # Add noise to the symbols
    noisy_qam_symbols = qam_symbols + noise
    return noisy_qam_symbols


# Generate 16QAM symbols
M = 16
num_symbols = 1000
qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)

#Signal to noise ratio in dB
#High signal to noise ratio is good, low is bad
snr_db = 15


# Add noise to QAM symbols
noisy_qam = add_awgn(qam_symbols, snr_db)

df = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
df.reset_index(inplace=True)

df.drop(['index'],axis=1,inplace=True)

#Getting data of different noise levels (data augmentation to prevent overfitting)
for i in range(15, 55, 5):
  M = 16
  num_symbols = 2000
  qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)
  snr_db = i
  noisy_qam = add_awgn(qam_symbols, snr_db)
  df1 = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
  df1.reset_index(inplace=True)
  df1.drop(['index'],axis=1,inplace=True)
  df = pd.concat([df, df1])

  #Plot Constellations (Before and After Noise)
  # plt.figure(figsize=(10,5))
  # plt.subplot(1,2,2)
  # plt.scatter(noisy_qam.real, noisy_qam.imag, alpha=0.5, label="Noisy")
  # plt.title(f"Signal to Noise Ratio = {snr_db} dB)")
  # plt.xlabel("In-phase (I)")
  # plt.ylabel("Quadrature (Q)")
  # plt.grid()

  #plt.show()
df = df.rename(columns={'features': 'binary','target':'complex'})
#To save as CSV
df.to_csv('dataWithNoise.csv', index = False)




Build Neural Network

X = Complex

y = Binary converted to Decimal

In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
#from sklearn.model_selection import cross_val_score #Remove this line

# Load data
df = pd.read_csv('dataWithNoise.csv')

# Keep target variable as integers (0-15)
y = df['binary'].values
y = [eval(yi) for yi in y] #convert string to list of ints
y = np.array(y)
y = y.reshape(-1, 4)

y = np.array([int("".join(str(bit) for bit in row), 2) for row in y])


# Features (unchanged)
X = df['complex'].values
X = [eval(xi) for xi in X] #convert string to list of ints
X = np.array([[x.real, x.imag] for x in X])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_norm = y_train / 15.0
y_test_norm = y_test / 15.0
# Build the regression model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,),
          kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu',
          kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='linear')  # Regression output
])


kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
y = np.round(y)
accuracies = [] #list to store the accuracy per fold
for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    y_train_norm = y_train / 15
    y_test_norm = y_test / 15

    earlyStopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        min_delta=0.001)

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    history = model.fit(X_train,
                        y_train,
                        epochs=20,
                        batch_size=32,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[earlyStopping])
    val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
    cv_scores.append(val_acc)


print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")



#Predict on the given values
X_pred = np.array([[3, -3],[1, -3],[3, -1],[-1, 3]])  # Convert X_pred to a NumPy array

predictions = model.predict(X_pred)
rounded_predictions = np.round(predictions).astype(int)
print(f"Predictions: {rounded_predictions}")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 55.0219 - mae: 6.8108 - val_loss: 13.7088 - val_mae: 3.5183
Epoch 2/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9.2649 - mae: 2.4672 - val_loss: 0.3203 - val_mae: 0.4403
Epoch 3/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.0388 - mae: 1.5736 - val_loss: 0.2458 - val_mae: 0.3416
Epoch 4/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.2696 - mae: 1.4012 - val_loss: 0.2250 - val_mae: 0.3333
Epoch 5/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.1783 - mae: 1.3783 - val_loss: 0.1939 - val_mae: 0.3096
Epoch 6/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.7367 - mae: 1.2795 - val_loss: 0.1799 - val_mae: 0.3078
Epoch 7/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.9900 - mae: 1.3414 - val_loss: 0.1345 - val_mae: 0.2465
Epoch 8/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.5672 - mae: 1.2353 - val_loss: 0.1070 - val_mae: 0.1993
Epoch 9/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.565

Cross-validation scores: [0.2552356719970703, 0.2215694636106491, 0.18147745728492737, 0.1756616085767746, 0.16920097172260284]
Mean CV Accuracy: 0.2006 (+/- 0.0329)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predictions: [[ 8]
 [12]
 [ 9]
 [ 6]]


In [ ]:
X_pred = np.array([[3, -3],[1, -3],[3, -1],[-1, 3]])  # Convert X_pred to a NumPy array

predictions = model.predict(X_pred)
print(f"Predictions: {predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[[1.]
 [1.]
 [1.]
 [1.]]
